<h2> Spectroscopic data reduction : spectra module

<h2> DIS

In [1]:
from pyvista import imred, tv, spectra
import pyvista.data
from importlib_resources import files
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

pyautogui does not seem to be available, disabling arrow key cursor moves


pyvista uses a display tool defined in the tv module. To use the interactive
display in a notebook, set the display to be an external display window, e.g. with 
<code>
%matplotlib qt
</code>
Instantiate a tv object, here we just call it t, but you could call it whatever you want!

In [2]:
%matplotlib qt
t=tv.TV()

The basic tool for basic image reduction is a Reducer object, defined in the imred module. Instantiate a reducer here. The main argument is an instrument name, which tells it to read a YAML configuration file for the specified instrument. We also give it an optional dir= argument to specify the default directory from which to read images, if a directory is not specified in subsequent commands that read images.

In [ ]:
red=imred.Reducer('DIS',dir='UT191019/DIS')

A main method of the reducer object is the reduce() method. Without any additional arguments, reduce() will read an image from disk, subtract the overscan (region(s) as determined from the instrument configuration file), compute an uncertainty array using the gain and readout noise from the instrument configuration file, and return a CCDData object with the data, uncertainty, and mask. 
<p>
To specify the input image, we could pass a string with the file name. If the string does not include a '/', it will read from the default input directory.
<p>
If the file can be identified with a unique integer, then you can just specify this number, which can be very convenient. This is turned into a character string using the formstr attribute define in the configuration file, which is used to search for the file to read.
<p>
We can display the image using the tv() method of our display tool, which can take as input a Data object, and numpy array, or a FITS HDU object.

In [4]:
a=red.reduce(28)

t.tv(a)

  Reading file: UT211031/echelle/HIP109807.0028.fits
  subtracting overscan:  1291.72


In [ ]:
star=red.reduce(24)
t.tv(star[0])
t.tv(star[1])

<h4> Calibration: make and apply flat field

If we add additional arguments to reduce(), we can add additional calibration steps. For example, to flat field the data, we would add a flat= keyword through which we give the reducer a flat field. To add a spatial bias subtraction, we would add a bias= keyword through which we give the reducer a superbias frame.
<br>
First, however, we have to make the calibration products, which is accomplished using the mkflat(), mkbias(), etc methods. These take as input a list of frames to be used to construct the master calibration frame (e.g.. superflat). For ARCES, we will keep the spectral shape of the flats for now.

Create biases and flats. Note that for flats, we have to do scattered light removal, which can be done on reduction of individual images, but since it is slow, we will do it on the combined flat. If we add the display= keyword, giving a display object, then the calibration frames will be displayed, showing each input frame relative to the master frame, so you can inspect and make sure that bad frames are not being included in the combination.

In [ ]:
tspec=imred.Reducer(inst='TSPEC',dir='UT191026/TSPEC',nfowler=8)
a=tspec.reduce(21)
dark=tspec.reduce(22)
t.clear()
t.tv(a)
a=a.subtract(dark)
t.tv(a)

Read and display a star spectral image. For ARCES, we do not apply a flat field here, since we only have flats with the orders, which can move around a bit. Instead, we will use "1D" flats later in the processing.

In [1]:
crstar=red.crrej(star,crbox='lacosmic',display=t,objlim=50)

NameError: name 'red' is not defined

<h4> Tracing and extraction

In [ ]:
trace=spectra.Trace(inst='DIS',channel=0,transpose=False)
vars(trace)

In [ ]:
trace.trace(star[0],[663],plot=t)
trace.write('DIS_blue_trace.fits')
trace.trace(star[1],[560],plot=t)
trace.write('DIS_red_trace.fits')

Sum up the arc lamp exposures, get the shift for the existing traces, and extract. Note that if you have a multiprocessor machine, you can specify number of threads to use for the extraction, which will speed things up (but the default threads=0 isn't too terrible).

In [ ]:
trace=spectra.Trace(file=files(pyvista.data).joinpath('DIS/DIS_blue_trace.fits'))
wav=spectra.WaveCal(file=files(pyvista.data).joinpath('DIS/DIS_blue_waves.fits'))

Do line identification based on previously identified lines, and wavelength fit.

In [ ]:
arcs=red.sum([1,2,3])

In [ ]:
wav.identify(arcec,plot=True,thresh=50) 

Now reduce an image

Get shift of traces, and extract. Alternatively, you could use a single call to retrace(), which will do the find() and then trace() using the shifted stored model as a starting guess.

Get the wavelengths for all pixels from the wavelength solution and plot extracted spectra.

In [114]:
wav.add_wave(imec)
plt.figure()
for row in range(len(imec.wave)) :
    plt.plot(imec.wave[row],imec.data[row])

Resample onto logarithmic wavelength grid and combine orders

In [ ]:
wnew=10**np.arange(3.5,4.0,5.5e-6)
comb=wav.scomb(imec.divide(flat1d),wnew,average=True,usemask=True)
plt.clf()
plt.plot(wnew,comb.data)

In [ ]:
t.tv(arcs[0])

In [ ]:
arcec=trace.extract(arcs[0],plot=t)
#wav=spectra.WaveCal()
#wav.identify(arcec,file='henear.dat',plot=True,wref=[3888,747],disp=1.823,thresh=50)
#wav.identify(arcec,file='henear.dat',plot=True,thresh=30)
#wav.write('DIS_blue_waves.fits')

In [ ]:
wav.identify(arcec,plot=True,thresh=30)

In [ ]:
trace=spectra.Trace(file=files(pyvista.data).joinpath('DIS/DIS_red_trace.fits'))
wav=spectra.WaveCal(file=files(pyvista.data).joinpath('DIS/DIS_red_waves.fits'))

In [ ]:
arcec=trace.extract(arcs[1],plot=t)
#wav=spectra.WaveCal()
#wav.identify(arcec,file='henear.dat',plot=True,inter=True,thresh=50)
#wav.identify(arcec,file='henear.dat',plot=True,thresh=50)
#wav.write('DIS_red_waves.fits')